## Imports

In [ ]:
# All imports here!
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

## Data Export

In [ ]:
# Read the data from the excel files
bva_results = pd.read_excel('bva_results.xlsx')
results_30m = pd.read_excel('30m_results.xlsx')
results_60m = pd.read_excel('60m_results.xlsx')

## Edusat-BVA Results

The benchmark on which the performance is tested is the SAT23 competition benchmark.
We will present graphs for two types of runs:
1. CNFs that required at most 30 minutes.
2. CNFs that required at least 30 minutes and at most 60 minutes.

In [ ]:
# The 30 minute database and the 60 minute database have the same columns
# Filter both of thems to only have the rows with the value of "Improvement Factor" != 1.
# After filtering the rows in both databse, create a new database with the rows that are in both databases.

results_30m = results_30m[results_30m['Improvement Factor'] != 1]
results_60m = results_60m[results_60m['Improvement Factor'] != 1]

mutual_values_in_30m = results_30m[results_30m['test'].isin(results_60m['test'])]
mutual_values_in_60m = results_60m[results_60m['test'].isin(results_30m['test'])]

# Add a check that checks if the mutual_values_in_30m and mutual_values_in_60m are empty or not
# If they are empty, print "No mutual values found"
# If they are not empty, print the number of mutual values found

if mutual_values_in_30m.empty and mutual_values_in_60m.empty:
    print("No mutual values found")
else:
    print(f"{len(mutual_values_in_30m)} mutual values found")

columns_to_convert = [
    'EDUSAT-BVA_clauses_reduced',
    'EDUSAT-BVA_clauses_added',
    'EDUSAT-BVA_clauses_deleted',
    'EDUSAT-BVA_auxiliary_variables'
]

for col in columns_to_convert:
    # If there's no value, put 0
    results_30m[col] = results_30m[col].fillna(0)
    results_60m[col] = results_60m[col].fillna(0)
    results_30m[col] = results_30m[col].astype(int)
    results_60m[col] = results_60m[col].astype(int)


In [ ]:
def survival_plot(results, title):
    fig, ax = plt.subplots()
    ax.set_xlabel('Number of tests solved')
    ax.set_ylabel('Time (s)')
    ax.set_title(title)
    
    edusat_filtered = results[results['EDUSAT_timeout'] == 0]
    # Sorting only the non-timeout results
    edusat_sorted = np.sort(edusat_filtered['EDUSAT_solve_time'])

    eudsat_bva_filtered = results[results['EDUSAT-BVA_timeout'] == 0]
    # Sorting only the non-timeout results
    edusat_bva_sorted = np.sort(eudsat_bva_filtered['EDUSAT-BVA_solve_time'])
    
    # Generating x values (1, 2, ..., N)
    edusat_x_values = np.arange(1, len(edusat_sorted) + 1)
    edusat_bva_x_values = np.arange(1, len(edusat_bva_sorted) + 1)
    
    ax.set_xticks(np.arange(0, len(edusat_sorted) + 1, 5))
    ax.set_yticks(np.arange(0, 4001, 600))
    
    ax.plot(edusat_x_values, edusat_sorted, marker='o', label='EDUSAT')
    ax.plot(edusat_bva_x_values, edusat_bva_sorted, marker='o', label='EDUSAT-BVA')
    ax.legend()
    plt.show()

survival_plot(results_30m, 'EDUSAT vs EDUSAT-BVA (30m Timeout)')
survival_plot(results_60m, 'EDUSAT vs EDUSAT-BVA (60m Timeout)')

In [ ]:
def scatter_plot(results, x_key='EDUSAT_solve_time', y_key='EDUSAT-BVA_solve_time', x_label=None, y_label=None, title='Scatter plot comparison', min_time_filter=None):
    fig, ax = plt.subplots()
    ax.set_xlabel(x_label if x_label else x_key.replace('_', ' ').title())
    ax.set_ylabel(y_label if y_label else y_key.replace('_', ' ').title())
    ax.set_title(title)
    
    x_times = results[x_key]
    y_times = results[y_key]

    # print(list(x_times))
    # print(list(y_times))
    
    # Apply filtering if min_time_filter is provided
    if min_time_filter is not None:
        mask = (x_times >= min_time_filter) | (y_times >= min_time_filter)
        x_times = x_times[mask]
        y_times = y_times[mask]
    
    assert len(x_times) == len(y_times)

    ax.scatter(x_times, y_times, alpha=0.6)
    max_time = max(x_times.max(), y_times.max())
    ax.plot([0, max_time], [0, max_time], linestyle='dashed', color='red', label='y=x')
    
    # Counting points above and below the line
    above_line = y_times > x_times
    below_line = y_times < x_times
    above_line_cnt = np.sum(above_line)
    below_line_cnt = np.sum(below_line)

    # Count edge cases EDUSAT got 1800 and EDUSAT-BVA got less and the opposite
    y_edge_cases_wins = (x_times == max_time) & (y_times < max_time)
    y_edge_cases_wins_cnt = np.sum(y_edge_cases_wins)
    x_edge_cases_wins = (y_times == max_time) & (x_times < max_time)
    x_edge_cases_wins_cnt = np.sum(x_edge_cases_wins)


    # print(list(above_line))
    # print(list(below_line))
    # # Find equal entries in above_line and below_line and print their index and their values in x_times and y_times
    # for i in range(len(list(x_times))):
    #     if list(above_line)[i] == list(below_line)[i]:
    #         print(f'Equal times: {list(x_times)[i]} and {list(y_times)[i]}')

    # Printing overall points on the plot
    ax.text(max_time * 0.8, max_time * 0.1, f'Total points: {len(x_times)}')
    
    # Printing counts instead of displaying on the graph
    print(f'{x_key} better: {above_line_cnt} out of {len(x_times)} (Unique wins: {x_edge_cases_wins_cnt})')
    print(f'{y_key} better: {below_line_cnt} out of {len(x_times)} (Unique wins: {y_edge_cases_wins_cnt})')

    # Printing equality
    # print(f'Equal times: {np.sum(x_times == y_times)} out of {len(x_times)}')
    
    ax.legend()
    plt.show()

In [ ]:
# Compare EDUSAT_solve_time with EDUSAT-BVA_solve_time
scatter_plot(results_30m, 'EDUSAT_solve_time', 'EDUSAT-BVA_solve_time', 
             'EDUSAT Solve Time (s)', 'EDUSAT-BVA Solve Time (s)', 
             'EDUSAT vs EDUSAT-BVA Solve (30m Timeout)')
# Compare EDUSAT_solve_time with EDUSAT-BVA_solve_time
scatter_plot(results_60m, 'EDUSAT_solve_time', 'EDUSAT-BVA_solve_time', 
             'EDUSAT Solve Time (s)', 'EDUSAT-BVA Solve Time (s)', 
             'EDUSAT vs EDUSAT-BVA Solve (60m Timeout)')

In [ ]:
# Commented out due to BVA-search time not being available in some 30m results

# # Count the occasions in which EDUSAT-BVA_timeout is 1, this means that EDUSAT-BVA_search_time wasn't reached
# print(f"EDUSAT-BVA_search_time wasn't reached {results_30m['EDUSAT-BVA_timeout'].sum()} times out of {len(results_30m)}")
# scatter_plot(results_30m, 'EDUSAT_solve_time', 'EDUSAT-BVA_search_time', 
#              'EDUSAT Search Time (s)', 'EDUSAT-BVA Search Time (s)', 
#              'EDUSAT vs EDUSAT-BVA Search (30m Timeout)')

# # Count the occasions in which EDUSAT-BVA_timeout is 1, this means that EDUSAT-BVA_search_time wasn't reached
# print(f"EDUSAT-BVA_search_time wasn't reached {results_60m['EDUSAT-BVA_timeout'].sum()} times out of {len(results_60m)}")
# scatter_plot(results_60m, 'EDUSAT_solve_time', 'EDUSAT-BVA_search_time', 
#              'EDUSAT Search Time (s)', 'EDUSAT-BVA Search Time (s)', 
#              'EDUSAT vs EDUSAT-BVA Search (60m Timeout)')

We'll present the same scatter plots while filtering all the results under 10 seconds!

In [ ]:
# Compare EDUSAT_solve_time with EDUSAT-BVA_solve_time
scatter_plot(results_30m, 'EDUSAT_solve_time', 'EDUSAT-BVA_solve_time', 
             'EDUSAT Solve Time (s)', 'EDUSAT-BVA Solve Time (s)', 
             'EDUSAT vs EDUSAT-BVA Solve (30m Timeout)', min_time_filter=180)
# Compare EDUSAT_solve_time with EDUSAT-BVA_solve_time
scatter_plot(results_60m, 'EDUSAT_solve_time', 'EDUSAT-BVA_solve_time', 
             'EDUSAT Solve Time (s)', 'EDUSAT-BVA Solve Time (s)', 
             'EDUSAT vs EDUSAT-BVA Solve (60m Timeout)', min_time_filter=180)

# Compare EDUSAT_solve_time with EDUSAT-BVA_search_time
# print(f"EDUSAT-BVA_search_time wasn't reached {results_30m['EDUSAT-BVA_timeout'].sum()} times out of {len(results_30m)}")
# scatter_plot(results_30m, 'EDUSAT_solve_time', 'EDUSAT-BVA_search_time', 
#              'EDUSAT Search Time (s)', 'EDUSAT-BVA Search Time (s)', 
#              'EDUSAT vs EDUSAT-BVA Search (30m Timeout)', min_time_filter=60)

# print(f"EDUSAT-BVA_search_time wasn't reached {results_60m['EDUSAT-BVA_timeout'].sum()} times out of {len(results_60m)}")
# scatter_plot(results_60m, 'EDUSAT_solve_time', 'EDUSAT-BVA_search_time', 
#              'EDUSAT Search Time (s)', 'EDUSAT-BVA Search Time (s)', 
#              'EDUSAT vs EDUSAT-BVA Search (60m Timeout)', min_time_filter=60)


We'll present below the most notable gains and regressions

In [ ]:
def print_rows(df, title):
    # Rename only the specified columns
    column_rename_map = {
        'EDUSAT_solve_time': 'EDUSAT (s)',
        'EDUSAT-BVA_solve_time': 'EDUSAT-BVA (s)',
        'EDUSAT-BVA_clauses_reduced': 'Clauses Reduced',
        'EDUSAT-BVA_auxiliary_variables': 'Extra Vars',
        'EDUSAT-BVA_preprocessing_time': 'BVA Preprocessing Time',
        'EDUSAT-BVA_search_time': 'BVA Search Time',
        'Improvement Factor': 'Improvement Factor'
    }

    df.rename(columns=column_rename_map, inplace=True)

    # Select only the renamed columns for display
    selected_columns = list(column_rename_map.values())

    display(
        df[selected_columns]
        .style.set_table_styles(
            [
                {"selector": "th", "props": [("text-align", "center")]},  # Center headers
                {"selector": "td", "props": [("text-align", "center")]}   # Center cell values
            ]
        )
        .set_caption(title)
        .set_properties(**{"text-align": "center"})  # Ensure pandas recognizes the alignment
    )

In [ ]:
top_30m = results_30m.nlargest(10, 'Improvement Factor').copy()
print_rows(top_30m, title='Top 10 Improvement Factors (30m Timeout)')

top_60m = results_60m.nlargest(10, 'Improvement Factor').copy()
print_rows(top_60m, title='Top 10 Improvement Factors (60m Timeout)')

In [ ]:
bottom_30m = results_30m.nsmallest(10, 'Improvement Factor').copy()
print_rows(bottom_30m, title='Bottom 10 Improvement Factors (30m Timeout)')

Finally, we'll present average times statistics

In [ ]:
def calculate_averages(results, title):
    edusat_avg_solve_time = results['EDUSAT_solve_time'].mean()
    edusat_bva_avg_search_time = results['EDUSAT-BVA_search_time'].mean()
    edusat_bva_avg_preprocessing_time = results['EDUSAT-BVA_preprocessing_time'].mean()
    edusat_bva_avg_solve_time = results['EDUSAT-BVA_solve_time'].mean()

    print(f'[{title} - #tests: {len(results)}]')
    print(f'    Average solve time for EDUSAT: {edusat_avg_solve_time:.2f} s')
    print(f'    Average search time for EDUSAT+BVA: {edusat_bva_avg_search_time:.2f} s')
    print(f'    Average preprocessing time for EDUSAT+BVA: {edusat_bva_avg_preprocessing_time:.2f} s')
    print(f'    Average solve time for EDUSAT+BVA: {edusat_bva_avg_solve_time:.2f} s')

# Example usage
calculate_averages(results_30m, '30m Timeout Results')
print('\n')
calculate_averages(results_60m, '60m Timeout Results')
